In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip3 install BitsAndBytes
!pip3 install accelerate

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

torch.cuda.empty_cache

model_id = "google/gemma-2b-it"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True, padding_side='left')


# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

In [4]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/LAiSERFiles/nlx_tx_sample_data_gwu.csv')

# Extract the 'description' column and convert it to a list of strings
text_list = df['description'].tolist()

# Print the list of texts
print(text_list)


["Req ID: 29534BR\n\nPOSITION SUMMARY\n\nThis position is open to any ASO location listed and will consider existing internal WAH employees in any area\n\nWill only consider External candidates located near the Phoenix office location listed in posting.\n\nThis trainer position will be a technical training position for the Aetna Medicaid Provider Data Services teams, including HPRA, CSS, RCAR, MPST, Testing and Provider Maintenance. This position will report directly into the Medicaid Service Operations team, not Learning and Performance.\n\nFundamental Components:\n-Responsible for the creation of the training curriculum from ground up for the Aetna Medicaid Provider Data Services teams, including, HPRA, CSS, Provider Maintenance, testing, RCAR and MPST teams.\n-Ability to understand the technical aspects of the current roles within these teams and convert them to learning components for new hire and up training opportunities.\n-Understanding of the Medicaid state web sites and how to

In [5]:
import torch

def get_completion_batch(queries: list, model, tokenizer, batch_size=2) -> list:
    device = "cuda:0"
    results = []

    prompt_template = """
    <start_of_turn>user
    Name all the skills present in the following description in a single list. Response should be in English and have only the skills, no other information or words. Skills should be keywords, each being no more than 3 words.
    Below text is the Description:

    {query}
    <end_of_turn>\n<start_of_turn>model
    """

    for i in range(0, len(queries), batch_size):
        batch = queries[i:i+batch_size]
        prompts = [prompt_template.format(query=query) for query in batch]

        encodeds = tokenizer(prompts, return_tensors="pt", add_special_tokens=True, padding=True, truncation=True)
        model_inputs = encodeds.to(device)

        with torch.no_grad():
            generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

        decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

        for full_output in decoded:
            # Extract only the model's response
            response = full_output.split("<start_of_turn>model")[-1].strip()
            results.append(response)

        # Clear CUDA cache after each batch
        torch.cuda.empty_cache()

        print(f"Processed batch {i//batch_size + 1}/{(len(queries)-1)//batch_size + 1}")

    return results

In [ ]:
raw_extracts = get_completion_batch(text_list, model, tokenizer)


print(raw_extracts[0])


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Processed batch 1/25


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Processed batch 2/25


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

Processed batch 5/25


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Processed batch 6/25


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Processed batch 7/25


In [6]:
import re

def fetch_model_output(response):
    # Find the content between the model start tag and the last <eos> tag
    pattern = r'<start_of_turn>model\s*<eos>(.*?)<eos>\s*$'
    match = re.search(pattern, response, re.DOTALL)

    if match:
        content = match.group(1).strip()

        # Split the content by lines and filter out empty lines
        lines = [line.strip() for line in content.split('\n') if line.strip()]

        # Extract skills (lines starting with '-')
        skills = [line[1:].strip() for line in lines if line.startswith('-')]

        return skills

In [ ]:
response = raw_extracts[0]
print(response)

In [7]:
torch.cuda.empty_cache

<function torch.cuda.memory.empty_cache() -> None>